In [1]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Functions import PredictiveAnalysis

import dash
from dash import html, dcc, ALL, MATCH, Output, Input, State, no_update, ctx, Patch
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc


In [2]:
# load the clean data frame
df = pd.read_csv('final.csv')
df.head()

,Date,Year,Month,CSENT,IPM,HOUSE,UNEMP,LRIR,SP500,SP500_Price,SP500_Rise
0,1979-01-31,1979,1,-13.859020,7.862464,-5.157233,5.9,-0.300399,11.966387,99.930000,1.0
1,1979-02-28,1979,2,-12.336892,7.786828,-8.596713,5.9,-0.671270,10.615806,96.279999,1.0
2,1979-03-31,1979,3,-13.197970,6.418676,-2.579853,5.8,-1.142366,13.877365,101.589996,1.0
3,1979-04-30,1979,4,-19.117647,2.997984,-13.425926,5.8,-1.135133,5.091398,101.760002,1.0
4,1979-05-31,1979,5,-17.852835,3.917937,-15.169195,5.6,-1.637674,1.892230,99.080002,1.0


# Dataset Creation
- Six types of moving averages ; from one to six months.
- Six types of dataset shifts; from one to six months.

In [3]:
"""
class PredictiveAnalysis_Test(PredictiveAnalysis):
    def __init__(self, df):
        super().__init__(df)
"""

'\nclass PredictiveAnalysis_Test(PredictiveAnalysis):\n    def __init__(self, df):\n        super().__init__(df)\n'

In [4]:
PA = PredictiveAnalysis(df)
PA.create_data(['CSENT', 'IPM', 'HOUSE', 'UNEMP', 'LRIR'], 'SP500', ma=[1,2,3,4,5,6], fp=[1,2,3,4,5,6], poly_d=1)

In [12]:
# check the format of each X and y matrix
print(PA.datasets['1MA_1FP']['X'][:5])
print(PA.datasets['1MA_1FP']['y'][:5])
print(PA.datasets['1MA_1FP']['y_cat'][:5])

[[ 1.         -1.07697478  1.02863002 -0.24524403 -1.02032316 -1.34994328]
 [ 1.         -0.97845534  1.01416757 -0.38013943 -1.02032316 -1.45430278]
 [ 1.         -1.0341884   0.75256061 -0.14415995 -1.09159136 -1.58686461]
 [ 1.         -1.41733832  0.09848356 -0.56953973 -1.09159136 -1.58482934]
 [ 1.         -1.3354736   0.2743895  -0.63791022 -1.23412778 -1.72623957]]
[[10.61580626]
 [13.87736507]
 [ 5.09139751]
 [ 1.89222954]
 [ 7.7253271 ]]
[[1]
 [1]
 [1]
 [1]
 [1]]


# Model Creation & Evaluation

- Four machine learning models
    1. Multiple Linear Regression
    3. Logistic Regression
    4. Classification and Regression Tree

<br>

- Evaluations
    - Regressions:
        - <b>Root Mean Square Error (RMSE)</b>: How much errors could occur between the predicted prices and the actual ones.
        - <b>Standard Error of Estimate (SE)</b>: How much variation could occur in the actual target based on the same condition of independent variables. 
        - <b>Coffeficient of Determination (R2)</b>: How well the regression model explains the variation of a target value.
        - <b>Adjusted R2</b>: R2 with the penalty for the number of independent variables.
        <br><br>
        
    - Classification
        - <b>Accuracy</b>: How the model can correctly predict the target values.
        - <b>Precision</b>: How the model can avoid false positives.
        - <b>Recall</b>: How the model can avoid false negatives.
        - <b>F1 Score</b>: How the model can balance precision and recall.
        - <b>ROC (Receiver Operating Characteristic)</b>: How the model can trade off between true positives and false positives.
        - <b>AUC (Area Under the ROC Curve)</b>: How the model can summarize the ROC curve.

### Multiple linear Regression
- Applying different scopes for parameter adjustments; [1,2,3,4,5,6]
- Feature selection by backward elimination for whole models.

In [9]:
# comparing model performance & backward elimination test
fig1_1, fig1_2, fig1_3 = PA.model_learning([1,3,6], model='LinR')

fig1_1.show()
fig1_2.show()
fig1_3.show()

In [10]:
# detailed performance
fig2_1, fig2_2 = PA.detail_perf(model='LinR', ma=1, fp=6, sc=1)
fig2_1.show()
fig2_2.show()

In [11]:
future = PA.futures['LinR']
for i in future.keys():
    for j in future[i].keys():
        if j == '1MA':
            yoy_mean = np.min(future[i][j], axis=0)
            print("YoY: ", yoy_mean)
            r = 12 - int(i[0])
            act_prices = df['SP500_Price'][-12:-r].values * (yoy_mean/100+1)
            print(act_prices)
            print('')

YoY:  [15.24544663]
[4462.28062197]

YoY:  [11.34064904 17.12459041]
[4311.08764087 4778.81225179]

YoY:  [21.64352171 15.26841803 20.34408793]
[4710.01280817 4703.07837486 4620.61125606]

YoY:  [13.75609013 13.80482224 13.73182425 14.90094577]
[4404.61303625 4643.36205486 4366.73339194 4684.05206732]

YoY:  [10.07139323 13.65203047 14.79525783 12.74256379 16.56810761]
[4261.94231015 4637.12798251 4407.56392436 4596.06346573 4627.92861041]

YoY:  [ 7.73274    11.50625791 12.21249451 12.38016438 12.62938223 13.92319243]
[4171.39012538 4549.5780992  4308.39872674 4581.28989103 4471.55530866
 4681.45720558]



### Logistic Regression

In [5]:
from sklearn.metrics import confusion_matrix, auc
from sklearn.linear_model import LogisticRegression

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [154]:
class PredictiveAnalysis_Test(PredictiveAnalysis):
    def __init__(self, df):
        super().__init__(df)
        self.colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
        self.titles = {
            'LinR': 'Linear Regression', 'LogR': 'Logistic Regression',
            'RMSE': 'RMSE', 'SE': 'SE', 'R2': 'R2', 'Adj-R2': 'Adj',
            'ACC': 'Accuracy', 'PRE': 'Precision', 'REC': 'Recall', 'F1': 'F1 score', 'AUC': 'Area Under ROC Curve'}
        
        self.results = {model: {} for model in ['LinR', 'LogR', 'CART']}  # theta, y_hat, sigma for each scope and dataset
        self.futures = {model: {} for model in ['LinR', 'LogR', 'CART']}  # predicted values for each 'fp'
        self.be_tests = {model: {'ma': {}, 'sc': {}} for model in ['LinR', 'LogR', 'CART']}  # backward elimination test results
        self.perf_df = {'LinR': pd.DataFrame(data=[], columns=['SC', 'MA', 'FP', 'RMSE', 'SE', 'R2', 'Adj-R2']),
                        'LogR': pd.DataFrame(data=[], columns=['SC', 'MA', 'FP', 'ACC', 'PRE', 'REC', 'F1', 'AUC']),
                        'CART': pd.DataFrame(data=[], columns=['SC', 'MA', 'FP', 'RMSE', 'SE', 'R2', 'Adj-R2'])}


    def model_learning2(self, scopes: list, model: str = '', eta_: float = 0.001, alpha_: float = 0.1, lambda_: float = 0.5, iter_: int = 100):
        """
        

        """
        # define variable
        self.sc_opts = scopes

        # set hyperparameters globally
        self.eta_ = eta_
        self.alpha_ = alpha_
        self.lambda_ = lambda_
        self.iter_ = iter_
 
        # set spaces
        self.be_tests[model]['ma'].update({ma: [] for ma in self.ma_opts})
        self.be_tests[model]['sc'].update({sc: [] for sc in scopes})
        self.futures[model] = {f'{k1}FP': {f'{k2}MA': np.zeros((len(scopes), k1)) for k2 in self.ma_opts} for k1 in self.fp_opts}

        idx = 0
        # each scope
        for i, sc in enumerate(scopes):
            # each dataset
            for j, d_key in enumerate(self.datasets.keys()):
                # acquire data
                data = self.datasets[d_key]
                if model == 'LinR':
                    theta, y_hat, error, future = self.linear_reg(X=data['X'], y=data['y'], t=120, sc=sc)

                elif model == 'LogR':
                    theta, y_hat, error, future = self.logistic_reg(X=data['X'], y=data['y_cat'], t=120, sc=sc)

                elif model == 'CART':
                    theta, y_hat, error, future = None, None, None, None

                else:
                    raise TypeError('Choose one of following model names: "LinR", "LogR", and "CART".')
                
                # get moving average and future performance values from dataset keys
                ma, fp = d_key.split('_')
                # store all data
                self.results[model][fp].update({f"{ma}_{sc}SC": {'theta': theta, 'y_hat': y_hat, 'error': error}})
                # store future values
                self.futures[model][fp][ma][i] = future

                # get test result of backward elimination
                y = data['y_cat'] if model == 'LogR' else data['y']
                be_test_df = self.evaluation(model, data['X'], y, 120, theta, y_hat, error)
                # retrienve only performance without any changes in each coefficient
                ma_int = int(re.findall(r'\d+', ma)[0])
                self.perf_df[model].loc[idx] = [sc, ma_int, fp] + list(be_test_df.iloc[0])
                # store its result as NDArray
                self.be_tests[model]['sc'][sc].append(np.array(be_test_df))
                ma_idx = self.ma_opts[j // len(self.fp_opts)]
                self.be_tests[model]['ma'][ma_idx].append(np.array(be_test_df))

                # increment idx
                idx += 1
        
        return self.compare_perf2(model)

        self.compere_perf_fig = self.compare_perf2(model)
        self.be_test_sc_fig = self.backward_elimination(model, 'sc')
        self.be_test_ma_fig = self.backward_elimination(model, 'ma')

        return self.compere_perf_fig, self.be_test_sc_fig, self.be_test_ma_fig


    def logistic_reg(self, X, y, t, sc):
        """
        Return the following five matrix (dtype: np.array)
        - "thetas"  -> parameters (intercept + coefficients) at each step
        - dictionary: predicted y values:
            - "cat" key: "y_preds_c" -> predicted labels at each step
            - "proba" ley: "y_preds_p" -> predicted probability at each step (nagetige & positive class)
        - "errors"  -> prediction errors (actual - predicted values); SSE
        - dictionary: future y values 
            - "cat": predicted labels at each step
            - "proba": probability of the positive class label

        Parameters:
        - "X": np.array -> independent variables
        - "y": np.array -> target variables (shouold be categorical)
        - "t": int -> number of data that were used for the initial parameter creation.
        - "sc": int -> scope of the latest data for parameter updates.

        Brief Steps:
        - Initialize all matries to store ithe ncrementally updated values.
        - Apply a given number of data ("t") to the mutiple linear regression (normal equation).
        - Define the initial parameters from the trained model.
        - At each step (total steps are len(y) - t):
            - Get a single pair of unfamilar data; both X and y.
            - Predict the target ("y_hats") based on the latest parameters("theta[i]").
            - Calculate the difference between actual and predicted values; "error[i]".
            - Update parameters for the next step ("theta[i+1]").
        """
        # define sigmoid function
        def sigmoid(h):
            return 1 / (1 + np.exp(-h))

        # calculate the gradient vector
        def gradient(X, y, theta, w, alpha_, lambda_):
            m = len(y)
            preds = sigmoid(np.dot(X, theta.reshape(-1,1)))
            grad = -np.dot(X.T, np.multiply(y - preds, w)) / m
            l1 = lambda_ * np.sign(theta)
            l2 = (1 - lambda_) * theta

            return grad.T + alpha_ * (l1 + l2)
        
        # initialize matrics to store values at each step
        thetas = np.zeros((len(y[t:])+1, 6))
        y_hats = np.zeros((len(y[120:]), 1))
        errors = np.zeros((len(y[t:]), 1))
        # initial training 
        logit = LogisticRegression(fit_intercept=False, class_weight='balanced')
        logit.fit(X[:t], y[:t].flatten())
        # set initial parameters
        thetas[0] = logit.coef_
        # define class weights
        weights = {u: len(X) / (2*np.bincount(y.flatten())[u]) for u in np.unique(y)}
        # define the vectorized function to converting class labels to weights
        vfunc_weights = np.vectorize(lambda x: weights[x])

        # start incremental learning
        for idx, i in enumerate(range(120, len(y), 1)):
            # apply sigmoid function
            y_hats[idx] = sigmoid(np.dot(X[i:i+1], thetas[idx]))
            # actual value
            y_act = y[i][0]
            # logistic loss
            y_proba = {0: 1-y_hats[idx], 1: y_hats[idx]}
            errors[idx] = -1 * weights[y_act] * np.log(y_proba[y_act])

            # subset pf X and y
            X_sub, y_sub = X[i+1-sc: i+1], y[i+1-sc: i+1]
            # update parameter
            theta_epoch = thetas[idx]
            for i in range(self.iter_):
                # get the partial derivative
                grad = gradient(X_sub, y_sub, theta_epoch, vfunc_weights(y_sub), self.alpha_, self.lambda_)
                # update the theta
                theta_epoch -= self.eta_* grad.flatten()

            thetas[idx+1] = theta_epoch

        # predict the future labels
        future = sigmoid(np.dot(X[len(y):], thetas[-1]))

        return thetas, y_hats, errors, future

    def compare_perf2(self, model: str):
         # define custom function
        def upper_error(x):
            return x.max() - x.mean()

        def lower_error(x):
            return x.mean() - x.min()

        # deine measures
        measures = list(self.perf_df[model].columns)[3:]
        # modity perf_df
        perf_df = self.perf_df[model].drop('FP', axis=1).groupby(['SC', 'MA']).agg(['mean', upper_error, lower_error])
        perf_df.columns = ['_'.join(col) for col in perf_df.columns]
        perf_df = perf_df.reset_index()
        perf_df['SC'] = perf_df['SC'].astype(str)

        # define figures
        figs = []
        # traversing all measures
        for i, ms in enumerate(measures):
            # define fifure
            fig = px.scatter(
                perf_df, x='MA', y=f'{ms}_mean', color='SC',
                error_y=f'{ms}_upper_error', error_y_minus=f'{ms}_lower_error'
            )

            fig.update_xaxes(title_text='Moving Averages', title_font={'color':'lightgrey'})
            fig.update_layout(
                height=300, width=400, template='plotly_dark',
                title=dict(text=self.titles[ms],  xanchor="center", x=0.50),
                showlegend=False, yaxis_title='', 
                margin=go.layout.Margin(t=50, l=30, r=30, b=50)
                ) 

            figs.append(fig)

            
        main_title = f'Comparing the {[model]} Model Results on Different Conditions'
        sub_title1 = f'<br><span {self.SUB_CSS}> -- Scopes: how many month of the latest data is used for parameter adjustments.</span>'
        sub_title2 = f'<br><span {self.SUB_CSS}> -- Error Bars: Showing mean, min and max of each measures among various future predictions.</span>'

        return figs

In [148]:
test = PredictiveAnalysis_Test(df)
test.create_data(['CSENT', 'IPM', 'HOUSE', 'UNEMP', 'LRIR'], 'SP500', ma=[1,2,3,4,5,6], fp=[1,2,3,4,5,6], poly_d=1)

In [155]:
figs = test.model_learning2(scopes=[1,2], model='LogR', iter_=1)
#figs2 = test.model_learning2(scopes=[1,2], model='LinR', iter_=1)

In [150]:
sc_opts = test.sc_opts
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

In [156]:
app = dash.Dash(__name__)

# Sep up each component
legends = dmc.ChipGroup(
    id={'func': 'compare_perf', 'obj': 'legend'},
    children=[
        dmc.Chip(
            children=str(sc),
            value=str(sc),
            variant="outline", 
            color=colors[i]
            ) for i, sc in enumerate(sc_opts)
    ],
    value=[str(sc) for sc in sc_opts],
    multiple=True,
    style={'display': 'flex', 'justifyContent': 'center'}
)

graphs = html.Div(
    dbc.Row(
        [
            dbc.Col(
                dbc.Card(
                    dbc.CardBody([
                        dcc.Graph(
                            id={'func': 'compare_perf', 'obj': 'fig', 'id': str(i)},
                            figure=fig,
                        )
                    ]),
                    style={'margin': '5px'}
                ),
                width='auto'
            ) for i, fig in enumerate(figs)
        ],
        style={'display': 'flex', 'overflowX': 'auto', 'width': '100%'},
    ),
    style={'maxWidth': '100vw'}
)

@app.callback(
    output=Output({'func': 'compare_perf', 'obj': 'fig', 'id': ALL}, 'figure'),
    inputs=Input({'func': 'compare_perf', 'obj': 'legend'}, 'value'),
    state=State({'func': 'compare_perf', 'obj': 'fig', 'id': ALL}, 'figure')
)
def update_visibility(value, fig):
    # Determine which input was triggered
    triggered_id = ctx.triggered_id
    if triggered_id:
        # define output
        outputs = []
        # get the checked value as set 
        checked = set(value)
        # traversing all figure data
        for f in fig:
            # define patch
            p = Patch()
            for i in range(len(f['data'])):
                p['data'][i].update({'visible': f['data'][i]['name'] in checked})
            
            outputs.append(p)
        
        return outputs
    
    else:
        return no_update


# Set up the Dash app layout 
app.layout = html.Div([
    legends, graphs
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [74]:
fig.show()

html.H3('Comparing the Logistic Regression Model Results on Different Conditions', 
            style={ 'color': 'white'}),
    html.Div(id='fixed-legend', style={'color': 'white'}, children=[
        dcc.Markdown("""
            - Scopes: how many months of the latest data is used for parameter adjustments.
            - Error Bars: Showing mean, min, and max of each measure among various future predictions.
        """)
    ]),

In [131]:
f1.show()

In [147]:
f2.show()

### Memo

In [4]:
import dash
from dash import Dash, html, dcc, Input, Output, State, ctx
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [20]:
# Assuming 'sample' is your DataFrame with the data you want to plot
# And it has columns 'X', 'Y1', 'Y2', ... for your data

# Sample data for three dataframes with similar structure
s1 = pd.DataFrame({
    'X': pd.date_range(start='1/1/2020', periods=100),
    'Y1': np.random.randn(100).cumsum(),
    'Y2': np.random.randn(100).cumsum(),
    'Y3': np.random.randn(100).cumsum()
})
s2 = pd.DataFrame({
    'X': pd.date_range(start='1/1/2020', periods=100),
    'Y1': np.random.randn(100).cumsum()*0.5,
    'Y2': np.random.randn(100).cumsum()*0.5,
    'Y3': np.random.randn(100).cumsum()*0.5
})  # Just for example, modify as needed

s3 = pd.DataFrame({
    'X': pd.date_range(start='1/1/2020', periods=100),
    'Y1': np.random.randn(100).cumsum()*2,
    'Y2': np.random.randn(100).cumsum()*2,
    'Y3': np.random.randn(100).cumsum()*2
})   # Just for example, modify as needed


# Define the number of graphs you want to create
fig = make_subplots(rows=1, cols=3, shared_yaxes=True)

# Define colors for traces to ensure consistency across subplots
colors = {'Y1': 'blue', 'Y2': 'red', 'Y3': 'green'}

for i, s in enumerate([s1, s2, s3], start=1):
    for col in ['Y1', 'Y2', 'Y3']:
        fig.add_trace(
            go.Scatter(
                x=s['X'],
                y=s[col],
                name=col,
                mode='lines+markers',
                marker=dict(color=colors[col]),
                showlegend=False, # Only the first subplot shows the legend,
                visible=True
            ),
            row=1, col=i
        )

# Update layout to Plotly's dark theme
fig.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font={'color': 'white'},
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    uirevision='constant' # keeps the user-selected legend state consistent across updates
)


app = dash.Dash(__name__)
# Set up the Dash app layout
app.layout = html.Div([
    html.Div(id='custom-legend', children=[
        html.Button('Y1', id='legend-y1', n_clicks=0),
        html.Button('Y2', id='legend-y2', n_clicks=0),
        html.Button('Y3', id='legend-y3', n_clicks=0)
    ], style={'display': 'flex', 'justifyContent': 'center'}),
    html.Div(style={'width': '600px', 'overflowX': 'scroll'}, children=[
        dcc.Graph(id='subplots-graph', figure=fig, style={'width': '1500px'})
    ])
])


@app.callback(
    output=Output('subplots-graph', 'figure'),
    inputs=dict(
        data=dict(
            y1=Input('legend-y1', 'n_clicks'),
            y2=Input('legend-y2', 'n_clicks'),
            y3=Input('legend-y3', 'n_clicks'),
        ),
    ),
    state=dict(fig=State('subplots-graph', 'figure'))
)
def update_graph_visibility(data, fig):
    # Determine which input was triggered
    triggered_id = ctx.triggered_id

    if triggered_id in {'legend-y1', 'legend-y2', 'legend-y3'}:
        # Get series name (e.g., 'Y1', 'Y2', 'Y3')
        series_name = triggered_id.split('-')[-1]
        # Toggle visibility
        visibility = False if data[series_name] % 2 == 1 else True
    
        # Update traces
        for trace in fig['data']:
            if trace['name'].lower() == series_name:
                trace['visible'] = visibility

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)
